In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = ""

In [ ]:
! pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.4 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",
)

In [ ]:
email_conversation = """
From: John (John@bikecorporation.me)
To: Kim (Kim@teddyinternational.me)
Subject: “ZENESIS” bike distribution cooperation and meeting schedule proposal
Dear Mr. Kim,

I am John, Senior Executive Director at Bike Corporation. I recently learned about your new bicycle model, "ZENESIS," through your press release. Bike Corporation is a company that leads innovation and quality in the field of bicycle manufacturing and distribution, with long-time experience and expertise in this field.

We would like to request a detailed brochure for the ZENESIS model. In particular, we need information on technical specifications, battery performance, and design aspects. This information will help us further refine our proposed distribution strategy and marketing plan.

Additionally, to discuss the possibilities for collaboration in more detail, I propose a meeting next Tuesday, January 15th, at 10:00 AM. Would it be possible to meet at your office to have this discussion?

Thank you.

Best regards,
John
Senior Executive Director
Bike Corporation
"""

### Output Oarser를 사용하지 않는 경우

In [ ]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessageChunk
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """
    Please extract the important parts of the following email.

    {email_conversation}
    """
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm


In [ ]:
def stream_response(response, return_output=False):
    """
    Streams the response from the AI model, processing and printing each chunk.

    This function iterates over each item in the 'response' iterable. If an item is an instance of AIMessageChunk, it extracts and prints the content.
    If the item is a string, it prints the string directly.
    Optionally, the function can return the concatenated string of all response chunks.

    Args:
    - response (iterable): An iterable of response chunks, which can be AIMessageChunk objects or strings.
    - return_output (bool, optional): If True, the function returns the concatenated response string. The default is False.

    Returns:
    - str: If `return_output` is True, the concatenated response string. Otherwise, nothing is returned.
    """
    for token in response:
        if isinstance(token, AIMessageChunk):
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            print(token, end="", flush=True)
    if return_output:
        return answer

answer = chain.stream({"email_conversation": email_conversation})

In [ ]:
output = stream_response(answer, return_output=True)

- **Sender:** John, Senior Executive Director at Bike Corporation
- **Recipient:** Kim at Teddy International
- **Subject:** "ZENESIS" bike distribution cooperation and meeting schedule proposal
- **Key Points:**
  - John learned about the "ZENESIS" bicycle model through a press release.
  - Bike Corporation is interested in a detailed brochure for the ZENESIS model, focusing on technical specifications, battery performance, and design aspects.
  - John proposes a meeting to discuss collaboration possibilities in more detail.
  - Proposed meeting date and time: Next Tuesday, January 15th, at 10:00 AM, at Kim's office.

In [ ]:
output

<generator object RunnableSequence.stream at 0x78ac77d6af10>

### Use OutPut Parser

In [35]:
class EmailSummary(BaseModel):
    person: str = Field(description="The sender of the email")
    email: str = Field(description="The email address of the sender")
    subject: str = Field(description="The subject of the email")
    summary: str = Field(description="A summary of the email content")
    date: str = Field(
        description="The meeting date and time mentioned in the email content"
    )

In [36]:
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [37]:
test_instruct=parser.get_format_instructions()


### prompt 제작

In [ ]:
prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant.

    QUESTION:
    {question}

    EMAIL CONVERSATION:
    {email_conversation}

    FORMAT:
    {format}
    """
)

In [ ]:
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "The sender of the email", "title": "Person", "type": "string"}, "email": {"description": "The email address of the sender", "title": "Email", "type": "string"}, "subject": {"description": "The subject of the email", "title": "Subject", "type": "string"}, "summary": {"description": "A summary of the email content", "title": "Summary", "type": "string"}, "date": {"description": "The me

In [41]:
chain = prompt | llm

In [42]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일을 요약.",
    }
)

In [43]:
output = stream_response(response, return_output=True)

```json
{
    "person": "John",
    "email": "John@bikecorporation.me",
    "subject": "“ZENESIS” bike distribution cooperation and meeting schedule proposal",
    "summary": "John, Senior Executive Director at Bike Corporation, is interested in the new bicycle model 'ZENESIS' from Teddy International. He requests a detailed brochure with technical specifications, battery performance, and design aspects to refine their distribution strategy and marketing plan. John proposes a meeting on Tuesday, January 15th, at 10:00 AM at Kim's office to discuss collaboration possibilities.",
    "date": "January 15th"
}
```

### with Parser

In [ ]:
chain = prompt | llm | parser

In [ ]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "Extract the main content of the email.",
    }
)

EmailSummary(person='John', email='John@bikecorporation.me', subject='“ZENESIS” bike distribution cooperation and meeting schedule proposal', summary="John, Senior Executive Director at Bike Corporation, is interested in the ZENESIS bike model and requests a detailed brochure with technical specifications, battery performance, and design aspects. He proposes a meeting to discuss potential collaboration on Tuesday, January 15th, at 10:00 AM at Kim's office.", date='January 15th')

In [ ]:
response.person

'John'